# MDBA Machine Learning Notebook

Annotating and training an object detector for automated counting of species

IMPORTANT NOTE: Parameters in params.yaml are used throughout the notebook.

The paths used throughout the notebook are relative and therefore the working directory must be the root of the machine learning directory. E.g.

<ul>
<li>Root</li>
    <ul>
    <li>data</li>    
    <li>inference</li>
    <li>models</li>
    <li>notebooks</li>
    <li>report</li>
    <li>requirements</li>
    <li>scripts</li>
    <li>test</li>    
    <li>utils</li>
    <li>params.yaml</li>
    <li>README.md</li>
    </ul>
</ul>


In [1]:
!python --version

Python 3.8.5


In [2]:
# Set working directory here
import sys, os
os.chdir(r'/home/azureuser/cloudfiles//code/Users/Ahsanul.Habib/WaterbirdCount/Drone-based-waterbird-counting')
os.getcwd()
!export PYTHONPATH=$rootfolder
sys.path.append(os.getcwd())

In [3]:
import argparse
import re
import csv
import cv2
import math
import codecs, json
from json import JSONEncoder
from tqdm import tqdm
from simple_colors import black
from pprint import pprint
from pathlib import Path
from random import seed, shuffle, sample
from shutil import copyfile, move
from subprocess import run, Popen
import yaml
from yaml import safe_load
import pylab
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import gc
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchmetrics.detection.mean_ap import MeanAveragePrecision

from utils.train_utils import collate_fn, get_transform, BirdDataset
from utils.Slicer import Slicer
from utils.utils import ensure_path
from utils.data_utils import CocoDataset
from utils.utils import str2bool
from utils.count import count_folder

from scripts.prepare_training_set import save_slices, cocofy_annotations, prepare_training_set
from scripts.split_raw_dataset import split_dataset, generate_training_set
from scripts.train_mdba import train_loop_fn, eval_loop_fn, train_mdba
from scripts.eval_mdba import groundTruth_annotations, draw_bboxes_save_results, plot_holdout_images_with_bboxes

/anaconda/envs/waterbirds/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
with open('./params.yaml', 'r') as params_file:
    params = yaml.safe_load(params_file)
    print(black('params.yml', ['italic']) + ' file is loaded.')

params.yml file is loaded.


# Split raw dataset 

This module takes raw images and a point file created in dotdotgoose to split them into appropriate size slices for training.

Slice size, as well as raw image size must be specified in the .yaml read in in the 2nd code chunk.

The slices are automatically placed into a new directory to be ingested by the 'prepare_training_set.py' module

In [7]:
# Initialise all required variables
raw_dir = Path(params['data']['raw_dir'])
project_name = params['data']['project_name']
project_folder = raw_dir / project_name
points_file = [file for file in os.listdir(project_folder) if file.endswith('.pnt')][0]
points_path = project_folder / points_file

holdouts_dir = Path(params['data']['holdouts_dir'])/(project_name+'_v1')
filenames = sorted(os.listdir(holdouts_dir))
no_annotation = [filename for filename in filenames if filename.lower().endswith('.jpg')]

trainval_dir = Path(params['data']['trainval_dir'])/project_name
holdouts_dir = Path(params['data']['holdouts_dir'])/project_name
nolabels_dir = Path(params['data']['nolabels_dir'])/project_name
trainval_output_dir = Path(params['slices']['trainval_dir'])/project_name/'sliced_images'
holdouts_output_dir = Path(params['slices']['holdouts_dir'])/project_name/'sliced_images'

# The .pnt file is necessary to prepare a training set.
# Otherwise the number of slices quickly becomes intractable.
points_file = [file for file in os.listdir(project_folder) if file.endswith('.pnt')][0]
points_path = project_folder / points_file
try:
    with open(points_path) as file:
        data = file.read()
        data = json.loads(data)
        points_dict = data.get('points')
except FileNotFoundError as err:
    print(f"{err}")
    raise
except NameError as err:
    print(f"{err}")
    print(f"Ensure that the .pnt file is properly formatted")
    raise

# First split the raw images into categories: trainval, holdouts, nolabels
holdouts_set, updated_points_dict = split_dataset(params, raw_dir / project_name, points_dict, trainval_dir, holdouts_dir, nolabels_dir, no_annotation)
points_dict = updated_points_dict
# Then, for the trainval and holdouts datasets, split them up and save the slices that have been identified by the .pnt file as containing objects. 
# This is done to filter out empty slices and reduce the labelling required for building a detector.
generate_training_set(params, points_dict, holdouts_set, trainval_dir, holdouts_dir, trainval_output_dir, holdouts_output_dir)

Filtering out images with missing annotation data: 100%|██████████| 205/205 [00:00<00:00, 1034695.93it/s]
Saving holdouts slices: 100%|██████████| 15/15 [02:48<00:00, 11.24s/it]


# Prepare training set

This script is used to scan through labels produced from label studio 
and prepare a dataset in the COCO format required for training.

In [8]:
prepare_training_set(params)

Adding negative samples: 100%|██████████| 88/88 [02:03<00:00,  1.40s/it]


# Train the Faster-RCNN model

In [ ]:
train_mdba(params)

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!
Epoch --> 1 / 25
-------------------------------


Training:   0%|          | 0/203 [00:00<?, ?it/s]/anaconda/envs/waterbirds/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Training: 100%|██████████| 203/203 [05:32<00:00,  1.64s/it]
/anaconda/envs/waterbirds/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


training Loss: 0.5053


Validation: 100%|██████████| 101/101 [00:43<00:00,  2.32it/s]


validation Loss: 0.4095
Epoch --> 2 / 25
-------------------------------


...

Training: 100%|██████████| 203/203 [05:42<00:00,  1.69s/it]


training Loss: 0.3103


Validation: 100%|██████████| 101/101 [00:43<00:00,  2.32it/s]


validation Loss: 0.3797
Epoch --> 7 / 25
-------------------------------


Training:  23%|██▎       | 46/203 [01:18<04:25,  1.69s/it]

# Evaluate trained model on the holdout set

If ground truth bounding boxes are provided, this module performs inference on the holdout image set and draws bounding boxes for ground truth data (green) and predicted data (blue/red).

The results json and images with bounding boxes drawn are save in: *./inference/results/DJI_202109281012_017_Mid_Lake_1010_70m_D2_50mm_10-lap_detections_available/*

In [13]:
slices_holdout_dir = Path(params['slices']['holdouts_dir'])/params['data']['project_name']/'sliced_images'
no_annotation_dir = Path(params['data']['holdouts_dir'])/(params['data']['project_name']+'_v1')
train_data_dir = Path(params['training']['train_data_dir'])/params['data']['project_name']/'sliced_images'
included_filenames = sorted(os.listdir(slices_holdout_dir))
included_files = [filename for filename in included_filenames if filename.lower().endswith('.jpg')]
labels_path = params['slices']['labels']

In [16]:
# This function transforms, or re-formats, the labels into the required format for inference.
with open(labels_path, 'r') as fr:
    ground_truth = groundTruth_annotations(params, slices_holdout_dir, json.loads(fr.read()), included_files)

Obtaining ground truth annotation for holdout set: 100%|██████████| 2201/2201 [00:01<00:00, 2151.74it/s]


In [17]:
gt_filepaths = [ground_truth[i]['filepath'] for i in range(len(ground_truth))]
filenames = [os.path.basename(gt_filepaths[i]) for i in range(len(gt_filepaths))]

In [18]:
groundtruths, inferences, images_with_bbox = draw_bboxes_save_results(params, filenames, ground_truth, slices_holdout_dir)

Drawing ground-truth and predicted boxes around birds in holdout set with available annotation data:   0%|          | 0/357 [00:00<?, ?it/s]/anaconda/envs/waterbirds/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Drawing ground-truth and predicted boxes around birds in holdout set with available annotation data: 100%|██████████| 357/357 [02:55<00:00,  2.04it/s]


In [ ]:
plot_holdout_images_with_bboxes(images_with_bbox)

# Calculate performance metric (mAP) for the trained model

In [63]:
metric = MeanAveragePrecision(box_format='xyxy', iou_type='bbox', iou_thresholds=list(np.linspace(0.5,0.95,91)), max_detection_thresholds=[3000])
metric.update(inferences, groundtruths)
result = metric.compute()

In [73]:
print(f"mAP@IoU=50%: {result['map_50'].item()}")
print(f"mAP@IoU=75%: {result['map_75'].item()}")

mAP@IoU=50%: 0.8480085134506226
mAP@IoU=75%: 0.6322376728057861
